In [1]:
import sys
sys.path.append('../WorldTrack')

In [129]:
import torch
from torch import FloatTensor, nn
import utils.vox
import utils.basic
from models.bevformernet import VanillaSelfAttention, SpatialCrossAttention
from models.decoder import Decoder

In [130]:
# setup camera settings
B = 1
S = 7
C = 3
H = 720
W = 1280
P = 4

# resolution
X = 360
Y = 120
Z = 4

In [131]:
# data input
rgb_cams = FloatTensor(size=(B,S,C,H,W)).random_().cuda()
pix_T_cams = FloatTensor(size=(B,S,P,P)).random_().cuda()
cams_T_global = FloatTensor(size=(B,S,P,P)).random_().cuda()
ref_T_global = FloatTensor(size=(B,P,P)).random_().cuda()

In [132]:
# # preprocess
# rgb_cams      (B,S,C,H,W) -- reshape -> rgb_cams_      (B*S,C,H,W)
# pix_T_cams    (B,S,P,P)   -- reshape -> pix_T_cams_    (B*S,P,P)
# cams_T_global (B,S,P,P)   -- reshape -> cams_T_global_ (B*S,P,P)   -- inv -> global_T_cams_ (B*S,P,P)
# ref_T_global  (B,P,P)     --------- reshape + mul global_T_cams_ ----------> ref_T_cams     (B*S,P,P)
__p = lambda x: utils.basic.pack_seqdim(x, B)
__u = lambda x: utils.basic.unpack_seqdim(x, B)
rgb_cams_ = __p(rgb_cams)  # B*S,3,H,W
pix_T_cams_ = __p(pix_T_cams)  # B*S,4,4
cams_T_global_ = __p(cams_T_global)  # B*S,4,4
global_T_cams_ = torch.inverse(cams_T_global_)  # B*S,4,4
ref_T_cams = torch.matmul(ref_T_global.repeat(S, 1, 1), global_T_cams_)  # B*S,4,4
cams_T_ref_ = torch.inverse(ref_T_cams)  # B*S,4,4

In [133]:
# rgb encoder
# pre-define data
rgb_cam_mean = torch.as_tensor([0.485, 0.456, 0.406]).reshape(1, 3, 1, 1).float().cuda()
rgb_cam_std = torch.as_tensor([0.229, 0.224, 0.225]).reshape(1, 3, 1, 1).float().cuda()
encoder = lambda z_rgb_cams: FloatTensor(size=(B*S,128,H//8,W//8)).random_().cuda()
# rgb_cams_ (B*S,C,H,W) -- compute (rgb_cams_ - mean) / std -> z_rgb_cams_ (B*S,C,H,W)
# z_rgb_cams_ (B*S,C,H,W) -- encode -> feat_cams_ (B*S,128,H/8,W/8) -- reshape -> feat_cams (B,S,128,H/8,W/8)
device = rgb_cams_.device
rgb_cams_ = (rgb_cams_ - rgb_cam_mean.to(device)) / rgb_cam_std.to(device)
feat_cams_ = encoder(rgb_cams_)  # B*S,128,H/8,W/8
_, C, Hf, Wf = feat_cams_.shape
feat_cams = __u(feat_cams_)  # B,S,C,Hf,Wf

In [164]:
# pre-define
vox_util = utils.vox.VoxelUtil(
    Y, Z, X,
    scene_centroid=torch.tensor([[0., 0., 0.]], dtype=torch.float32),
    bounds=[0, 1440, 0, 480, 0, 200]
)

# compute the image locations (no flipping for now)
xyz_mem_ = utils.basic.gridcloud3d(
    B*S, Y, Z, X, 
    norm=False, 
    device=rgb_cams.device
)  # B*S, Z*Y*X, 3
print(xyz_mem_.shape)

xyz_ref_ = vox_util.Mem2Ref(
    xyz_mem_,
    Y, Z, X,
    assert_cube=False
)
print(xyz_ref_.shape)

xyz_cams_ = utils.geom.apply_4x4(cams_T_ref_, xyz_ref_)
print(xyz_cams_.shape)
xy_cams_ = utils.geom.camera2pixels(xyz_cams_, pix_T_cams_)  # B*S, N, 2
print(xy_cams_.shape)

torch.Size([7, 172800, 3])
torch.Size([7, 172800, 3])
torch.Size([7, 172800, 3])
torch.Size([7, 172800, 2])


In [167]:
X, Y, Z

(360, 120, 4)

In [169]:
172800/X/Y/Z

1.0

In [135]:
# pre-define
z_sign = 1
latent_dim=128
feat2d_dim=128

# bev coords project to pixel level and normalized  S,B,Y*X,Z,2
reference_points_cam = xy_cams_.reshape(B, S, Y, Z, X, 2).permute(1, 0, 2, 4, 3, 5).reshape(S, B, Y * X, Z, 2)
reference_points_cam[..., 0:1] = reference_points_cam[..., 0:1] / float(W)
reference_points_cam[..., 1:2] = reference_points_cam[..., 1:2] / float(H)
cam_x = xyz_cams_[..., 2].reshape(B, S, Y, Z, X, 1).permute(1, 0, 2, 4, 3, 5).reshape(S, B, Y * X, Z, 1)
bev_mask = ((reference_points_cam[..., 1:2] >= 0.0)
            & (reference_points_cam[..., 1:2] <= 1.0)
            & (reference_points_cam[..., 0:1] <= 1.0)
            & (reference_points_cam[..., 0:1] >= 0.0)
            & (z_sign * cam_x >= 0.0)
            ).squeeze(-1)  # S,B,Y*X,Z valid point or not

In [136]:
# pre-define
bev_keys_W = nn.Linear(feat2d_dim, latent_dim).cuda()
bev_queries_W = nn.Parameter(0.1 * torch.randn(latent_dim, Y, X)).cuda()  # C, Y, X
bev_queries_pos_W = nn.Parameter(0.1 * torch.randn(latent_dim, Y, X)).cuda()  # C, Y, X

# self-attention prepare
bev_queries = bev_queries_W.clone().unsqueeze(0).repeat(B, 1, 1, 1).reshape(B, latent_dim, -1) \
    .permute(0, 2, 1)  # B, Y*X, C
bev_queries_pos = bev_queries_W.clone().unsqueeze(0).repeat(B, 1, 1, 1) \
    .reshape(B, latent_dim, -1).permute(0, 2, 1)  # B, Y*X, C

In [137]:
# cross-attention prepare
bev_keys = feat_cams.reshape(B, S, C, Hf * Wf).permute(1, 3, 0, 2)  # S, Hf*Wf, B, C
bev_keys = bev_keys_W(bev_keys)
spatial_shapes = bev_queries.new_zeros([1, 2]).long()
spatial_shapes[0, 0] = Hf
spatial_shapes[0, 1] = Wf

In [ ]:
# pre-define
num_layers = 6
num_classes = 1
self_attn_layers = nn.ModuleList([
    VanillaSelfAttention(dim=latent_dim) for _ in range(num_layers)
]).cuda()  # deformable self attention
norm1_layers = nn.ModuleList([
    nn.LayerNorm(latent_dim) for _ in range(num_layers)
]).cuda()
cross_attn_layers = nn.ModuleList([
    SpatialCrossAttention(dim=latent_dim) for _ in range(num_layers)
]).cuda()
norm2_layers = nn.ModuleList([
    nn.LayerNorm(latent_dim) for _ in range(num_layers)
]).cuda()
ffn_dim = 512
ffn_layers = nn.ModuleList([
    nn.Sequential(nn.Linear(latent_dim, ffn_dim), nn.ReLU(), nn.Linear(ffn_dim, latent_dim)) for _ in
    range(num_layers)
]).cuda()
norm3_layers = nn.ModuleList([
    nn.LayerNorm(latent_dim) for _ in range(num_layers)
]).cuda()

bev_temporal = nn.Sequential(
    nn.Conv2d(latent_dim * 2, latent_dim, kernel_size=3, padding=1),
    nn.InstanceNorm2d(latent_dim), nn.ReLU(),
    nn.Conv2d(latent_dim, latent_dim, kernel_size=1),
).cuda()

# Decoder
decoder = Decoder(
    in_channels=latent_dim,
    n_classes=num_classes,
    feat2d=feat2d_dim,
)

In [160]:
self_attn_layers[0].to('cuda')

VanillaSelfAttention(
  (dropout): Dropout(p=0.5, inplace=False)
  (deformable_attention): MSDeformAttn(
    (sampling_offsets): Linear(in_features=128, out_features=64, bias=True)
    (attention_weights): Linear(in_features=128, out_features=32, bias=True)
    (value_proj): Linear(in_features=128, out_features=128, bias=True)
    (output_proj): Linear(in_features=128, out_features=128, bias=True)
  )
  (output_proj): Linear(in_features=128, out_features=128, bias=True)
)

In [162]:
self_attn_layers[0].to('cuda')(bev_queries, Y, X, bev_queries_pos)

RuntimeError: ms_deform_attn_impl_forward: implementation for device cuda:0 not found.

Exception raised from Dispatch at C:\Users\PoraRee\AppData\Local\Temp\pip-install-672wsqnj\mmcv_cb600b9f4be24b059bad090755f46a85\mmcv\ops\csrc\common\pytorch_device_registry.hpp:122 (most recent call first):
00007FFEEEA891D900007FFEEEA89130 c10.dll!c10::Error::Error [<unknown file> @ <unknown line number>]
00007FFEEEA87A6A00007FFEEEA87A10 c10.dll!c10::detail::torchCheckFail [<unknown file> @ <unknown line number>]
00007FFD3CF278CD00007FFD3CEC65F0 _ext.cp39-win_amd64.pyd!c10::ivalue::Object::operator= [<unknown file> @ <unknown line number>]
00007FFD3CF27E4500007FFD3CEC65F0 _ext.cp39-win_amd64.pyd!c10::ivalue::Object::operator= [<unknown file> @ <unknown line number>]
00007FFD3CF555B800007FFD3CF353E0 _ext.cp39-win_amd64.pyd!PyInit__ext [<unknown file> @ <unknown line number>]
00007FFD3CF5564400007FFD3CF353E0 _ext.cp39-win_amd64.pyd!PyInit__ext [<unknown file> @ <unknown line number>]
00007FFD3CF3430A

In [142]:
for i in range(num_layers):
    print("SELF ATTN")
    # self attention within the features (B, Y*X, C)
    bev_queries = self_attn_layers[i](bev_queries, Y, X, bev_queries_pos)

    print("NORM1")
    # normalize (B, Y*X, C)
    bev_queries = norm1_layers[i](bev_queries)

    # cross attention into the images
    bev_queries = cross_attn_layers[i](bev_queries, bev_keys, bev_keys,
                                            query_pos=bev_queries_pos,
                                            reference_points_cam=reference_points_cam,
                                            spatial_shapes=spatial_shapes,
                                            bev_mask=bev_mask)

    # normalize (B, N, C)
    bev_queries = norm2_layers[i](bev_queries)

    # feedforward layer (B, N, C)
    bev_queries = bev_queries + ffn_layers[i](bev_queries)

    # normalize (B, N, C)
    bev_queries = norm3_layers[i](bev_queries)

SELF ATTN


RuntimeError: ms_deform_attn_impl_forward: implementation for device cuda:0 not found.

Exception raised from Dispatch at C:\Users\PoraRee\AppData\Local\Temp\pip-install-672wsqnj\mmcv_cb600b9f4be24b059bad090755f46a85\mmcv\ops\csrc\common\pytorch_device_registry.hpp:122 (most recent call first):
00007FFEEEA891D900007FFEEEA89130 c10.dll!c10::Error::Error [<unknown file> @ <unknown line number>]
00007FFEEEA87A6A00007FFEEEA87A10 c10.dll!c10::detail::torchCheckFail [<unknown file> @ <unknown line number>]
00007FFD3CF278CD00007FFD3CEC65F0 _ext.cp39-win_amd64.pyd!c10::ivalue::Object::operator= [<unknown file> @ <unknown line number>]
00007FFD3CF27E4500007FFD3CEC65F0 _ext.cp39-win_amd64.pyd!c10::ivalue::Object::operator= [<unknown file> @ <unknown line number>]
00007FFD3CF555B800007FFD3CF353E0 _ext.cp39-win_amd64.pyd!PyInit__ext [<unknown file> @ <unknown line number>]
00007FFD3CF5564400007FFD3CF353E0 _ext.cp39-win_amd64.pyd!PyInit__ext [<unknown file> @ <unknown line number>]
00007FFD3CF3430A

In [ ]:
shapelist

[2, 7, 3, 720, 1280]